## LangChain: Memory
### Outline
- ConversationBufferMemory
- ConversationBufferWindowMemory
- ConversationTokenBufferMemory
- ConversationSummaryMemory

In [ ]:
from langchain.chains import ConversationChain
from langchain.memory import ConversationBufferMemory
from langchain_openai import ChatOpenAI

chat = ChatOpenAI(
    model='deepseek-chat', 
    openai_api_key='sk-098aba2c0cf84951a8e796d406fcb055', 
    openai_api_base = 'https://api.deepseek.com',
    max_tokens=1024
)

AIMessage(content='Hello, Bob! How can I assist you today?', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 11, 'prompt_tokens': 9, 'total_tokens': 20, 'completion_tokens_details': None, 'prompt_tokens_details': None, 'prompt_cache_hit_tokens': 0, 'prompt_cache_miss_tokens': 9}, 'model_name': 'deepseek-chat', 'system_fingerprint': 'fp_1bcb2de9ac', 'finish_reason': 'stop', 'logprobs': None}, id='run-3d7571b7-f7ee-4e57-9c1c-ba613eb1954b-0', usage_metadata={'input_tokens': 9, 'output_tokens': 11, 'total_tokens': 20, 'input_token_details': {}, 'output_token_details': {}})

#### deprecated

In [14]:
memory = ConversationBufferMemory()
conversation = ConversationChain(
    llm=chat, 
    memory = memory,
    verbose=True
)
conversation.predict(input="Hi, my name is Andrew")
conversation.predict(input="What is 1+1?")
conversation.predict(input="What is my name?")
print(memory.buffer)
memory.load_memory_variables({})
memory = ConversationBufferMemory()
memory.save_context({"input": "Hi"}, 
                    {"output": "What's up"})
print(memory.buffer)
memory.load_memory_variables({})
memory.save_context({"input": "Not much, just hanging"}, 
                    {"output": "Cool"})
memory.load_memory_variables({})



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:

Human: Hi, my name is Andrew
AI:

> Finished chain.


> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
Human: Hi, my name is Andrew
AI: Hello, Andrew! It's great to meet you. How can I assist you today? Do you have any questions or topics you'd like to discuss?
Human: What is 1+1?
AI:

> Finished chain.


> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly co

{'history': "Human: Hi\nAI: What's up\nHuman: Not much, just hanging\nAI: Cool"}

#### simple

For more details, please refer to [RunnableWithMessageHistory ](https://python.langchain.com.cn/docs/expression_language/how_to/message_history)

In [ ]:
from langchain_core.chat_history import InMemoryChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory
from langchain_openai import ChatOpenAI


store = {}  # memory is maintained outside the chain

def get_session_history(session_id: str) -> InMemoryChatMessageHistory:
    if session_id not in store:
        store[session_id] = InMemoryChatMessageHistory()
    return store[session_id]

chain = RunnableWithMessageHistory(chat, get_session_history)
chain.invoke(
    "Hi I'm Bob.",
    config={"configurable": {"session_id": "1"}}
)  # session_id determines thread

chain.invoke(input="What is 1+1?",config={"configurable": {"session_id": "1"}})
chain.invoke(input="What is my name?",config={"configurable": {"session_id": "1"}})

for index, messages in store.items():
    for data in messages:
        for human, ai in zip(data[1][:-1:2], data[1][1::2]):
            print(human.content)
            print(ai.content)

## result
# store["1"] = InMemoryChatMessageHistory(
#     messages=[
#         HumanMessage(content="Hi I'm Bob.", additional_kwargs={}, response_metadata={}), AIMessage(content='Hello, Bob! How can I assist you today?', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 11, 'prompt_tokens': 9, 'total_tokens': 20, 'completion_tokens_details': None, 'prompt_tokens_details': None, 'prompt_cache_hit_tokens': 0, 'prompt_cache_miss_tokens': 9}, 'model_name': 'deepseek-chat', 'system_fingerprint': 'fp_1bcb2de9ac', 'finish_reason': 'stop', 'logprobs': None}, id='run-79dba4f6-8ab3-49f7-ab20-f2fbd289e703-0', usage_metadata={'input_tokens': 9, 'output_tokens': 11, 'total_tokens': 20, 'input_token_details': {}, 'output_token_details': {}}), 
#         HumanMessage(content='What is 1+1?', additional_kwargs={}, response_metadata={}), AIMessage(content='1 + 1 equals **2**. 😊', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 10, 'prompt_tokens': 30, 'total_tokens': 40, 'completion_tokens_details': None, 'prompt_tokens_details': None, 'prompt_cache_hit_tokens': 0, 'prompt_cache_miss_tokens': 30}, 'model_name': 'deepseek-chat', 'system_fingerprint': 'fp_1bcb2de9ac', 'finish_reason': 'stop', 'logprobs': None}, id='run-f0111795-29fa-4c54-9a0f-3c095019d31f-0', usage_metadata={'input_tokens': 30, 'output_tokens': 10, 'total_tokens': 40, 'input_token_details': {}, 'output_token_details': {}}), 
#         HumanMessage(content='What is my name?', additional_kwargs={}, response_metadata={}), AIMessage(content='Your name is Bob. 😊', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 7, 'prompt_tokens': 48, 'total_tokens': 55, 'completion_tokens_details': None, 'prompt_tokens_details': None, 'prompt_cache_hit_tokens': 0, 'prompt_cache_miss_tokens': 48}, 'model_name': 'deepseek-chat', 'system_fingerprint': 'fp_1bcb2de9ac', 'finish_reason': 'stop', 'logprobs': None}, id='run-63619cdb-ea68-482c-8897-a5569acf4a0b-0', usage_metadata={'input_tokens': 48, 'output_tokens': 7, 'total_tokens': 55, 'input_token_details': {}, 'output_token_details': {}})])

AIMessage(content='Your name is Bob.', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 5, 'prompt_tokens': 46, 'total_tokens': 51, 'completion_tokens_details': None, 'prompt_tokens_details': None, 'prompt_cache_hit_tokens': 0, 'prompt_cache_miss_tokens': 46}, 'model_name': 'deepseek-chat', 'system_fingerprint': 'fp_1bcb2de9ac', 'finish_reason': 'stop', 'logprobs': None}, id='run-dea17ac2-0d0e-49da-b383-40bd0a492e0d-0', usage_metadata={'input_tokens': 46, 'output_tokens': 5, 'total_tokens': 51, 'input_token_details': {}, 'output_token_details': {}})